In [1]:
from Bio import SeqIO
import sys
from optparse import OptionParser
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
%matplotlib inline
import matplotlib.colors as mcolors
import matplotlib.cm as cm

In [2]:
def GC_content_window(s):
    
    """
    Return GC content of input sequence
    """

    gc = sum(s.count(x) for x in ['G','C','g','c','S','s'])
    sl = sum(s.count(x) for x in ['A','a','T','t','G','C','g','c','S','s'])
    try:
        gc_content = gc/float(sl)
    except ZeroDivisionError:
        gc_content = 0
    return round(gc_content,4) 

def GC_skew_window(s):
    
    """
    Return GC skew of a sequence
    """

    g = s.count('G')+s.count('g')
    c = s.count('C')+s.count('c')

    try:
        skew = (g-c)/float(g+c)
    except ZeroDivisionError:
        skew = 0
    return round(skew,4)

def AT_skew_window(s):
    
    """
    Return GC skew of a sequence
    """

    a = s.count('A')+s.count('a')
    t = s.count('T')+s.count('t')

    try:
        skew = (a-t)/float(a+t)
    except ZeroDivisionError:
        skew = 0
    return round(skew,4)

def AG_content_window(s):

    """
    Return AG content of a sequence
    """
    
    ag = sum(s.count(x) for x in ['A','G','a','g', 'R', 'r'])
    sl = sum(s.count(x) for x in ['A','a','T','t','G','C','g','c','S','s'])
    try:
        ag_content = ag/float(sl)
    except ZeroDivisionError:
        ag_content = 0
    return round(ag_content,4)

def GC_content_window_NR(s):
    
    """
    Return GC content of input sequence
    """

    gc = sum(s.count(x) for x in ['G','C','S'])
    sl = sum(s.count(x) for x in ['A','T','G','C','S'])
    try:
        gc_content = gc/float(sl)
    except ZeroDivisionError:
        gc_content = 0
    return round(gc_content,4) 

def GC_skew_window_NR(s):
    
    """
    Return GC skew of a sequence
    """

    g = s.count('G')
    c = s.count('C')

    try:
        skew = (g-c)/float(g+c)
    except ZeroDivisionError:
        skew = 0
    return round(skew,4)

def AT_skew_window_NR(s):
    
    """
    Return GC skew of a sequence
    """

    a = s.count('A')
    t = s.count('T')

    try:
        skew = (a-t)/float(a+t)
    except ZeroDivisionError:
        skew = 0
    return round(skew,4)

def AG_content_window_NR(s):

    """
    Return AG content of a sequence
    """
    
    ag = sum(s.count(x) for x in ['A','G','R'])
    sl = sum(s.count(x) for x in ['A','T','G','C','S'])
    try:
        ag_content = ag/float(sl)
    except ZeroDivisionError:
        ag_content = 0
    return round(ag_content,4)

def GC_content_window_R(s):
    
    """
    Return GC content of input sequence
    """

    gc = sum(s.count(x) for x in ['g','c','s'])
    sl = sum(s.count(x) for x in ['a','t','g','c','s'])
    try:
        gc_content = gc/float(sl)
    except ZeroDivisionError:
        gc_content = 0
    return round(gc_content,4) 

def GC_skew_window_R(s):
    
    """
    Return GC skew of a sequence
    """

    g = s.count('g')
    c = s.count('c')

    try:
        skew = (g-c)/float(g+c)
    except ZeroDivisionError:
        skew = 0
    return round(skew,4)

def AT_skew_window_R(s):
    
    """
    Return GC skew of a sequence
    """

    a = s.count('a')
    t = s.count('t')

    try:
        skew = (a-t)/float(a+t)
    except ZeroDivisionError:
        skew = 0
    return round(skew,4)

def AG_content_window_R(s):

    """
    Return AG content of a sequence
    """
    
    ag = sum(s.count(x) for x in ['a','g', 'r'])
    sl = sum(s.count(x) for x in ['a','t','g','c','s'])
    try:
        ag_content = ag/float(sl)
    except ZeroDivisionError:
        ag_content = 0
    return round(ag_content,4)

def null_value_window(s):
    """
    return number of Ns in a sequence
    """
    n = sum(s.count(x) for x in ['N'])
    n_percent = n/float(len(s))
    return round(n_percent, 4)

def hex_to_rgba(hex_string):
    rgb = mcolors.hex2color(hex_string)
    return rgb+(1,)

In [3]:
window = 20000
step = 20000
nThreshold = 0.5
graphShape = (10,12)

color_mapping = {"A": "#A6CDE3", 
                  "B": "#089E78", 
                  "C": "#F19799", 
                  "D": "#F7F09B", 
                  "E": "#AF5B2A",
                  "F": "#DA6227", 
                  "G": "#7671B0", 
                  "H": "#E42C8A", 
                  "I": "#65A744", 
                  "J": "#C9B2D3",
                  "K": "#E6AB22", 
                  "L": "#A4782B", 
                  "M": "#90CDC3", 
                  "N": "#686868", 
                  "O": "#EF7E25",
                  "P": "#BDB9DA", 
                  "Q": "#FED832", 
                  "R": "#00008B",
                  "S": "#FED832"}
"""
numChromosomes = 19
filename = 'input_data/genomes/Bfl.fna'
gtffile = 'input_data/gene_rows/Bfl.gtf'
SCfile = 'input_data/gene_rows_SC/Bfl_coordinates.tsv'
"""

"\nnumChromosomes = 19\nfilename = 'input_data/genomes/Bfl.fna'\ngtffile = 'input_data/gene_rows/Bfl.gtf'\nSCfile = 'input_data/gene_rows_SC/Bfl_coordinates.tsv'\n"

In [4]:
"""
records = SeqIO.parse(filename, 'fasta')
data = [(record.id, str(record.seq), len(record.seq)) for record in records]
seqobj = pd.DataFrame(data, columns=['id', 'seq', 'seqlen'])
seqobj = seqobj.sort_values('seqlen', ascending=False).head(numChromosomes)

columns = ["sequence", "source", "feature", "start", "end", "score", "strand", "frame", "attribute"]
gtf_dataframe = pd.read_csv(gtffile, sep = '\t', comment = '#', header = None, names = columns, dtype={'start': int, 'end': int})

SCcolumns = ["index", "Junk", "sequence", "start", "end", "color_key"]
SC_dataframe = pd.read_csv(SCfile, sep = '\t', comment = '#', header = None, names = SCcolumns, dtype={'start': int, 'end': int})
SC_dataframe = SC_dataframe[SC_dataframe['color_key'].isin(["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S"])]
SC_dataframe['color'] = SC_dataframe['color_key'].map(color_mapping)
"""


'\nrecords = SeqIO.parse(filename, \'fasta\')\ndata = [(record.id, str(record.seq), len(record.seq)) for record in records]\nseqobj = pd.DataFrame(data, columns=[\'id\', \'seq\', \'seqlen\'])\nseqobj = seqobj.sort_values(\'seqlen\', ascending=False).head(numChromosomes)\n\ncolumns = ["sequence", "source", "feature", "start", "end", "score", "strand", "frame", "attribute"]\ngtf_dataframe = pd.read_csv(gtffile, sep = \'\t\', comment = \'#\', header = None, names = columns, dtype={\'start\': int, \'end\': int})\n\nSCcolumns = ["index", "Junk", "sequence", "start", "end", "color_key"]\nSC_dataframe = pd.read_csv(SCfile, sep = \'\t\', comment = \'#\', header = None, names = SCcolumns, dtype={\'start\': int, \'end\': int})\nSC_dataframe = SC_dataframe[SC_dataframe[\'color_key\'].isin(["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S"])]\nSC_dataframe[\'color\'] = SC_dataframe[\'color_key\'].map(color_mapping)\n'

In [5]:
"""
#Add in remove repeats
fig, ax = plt.subplots(figsize=(8,8))
colors = cm.viridis(np.linspace(0,1, numChromosomes))
genome = ""
gcR = []
gc = []
j=0
for index, row in seqobj.iterrows(): 
        pos_array = []
        gc_content_value_array = []
        gc_content_value_array_NR = []
        gc_content_value_array_R = []
    
        n_percent_array = []
        name = row['id']
        seq = row['seq']
        for i in range(0,len(seq),step):
            subseq = seq[i:i+window]
            gc_content = (GC_content_window(subseq))
            gc_content_NR = (GC_content_window_NR(subseq))
            gc_content_R = (GC_content_window_R(subseq))
            n_percent = (null_value_window(subseq))
            #start = (i + 1 if (i+1<=len(seq)) else i)
            #end = ( i + step if (i+ step<=len(seq)) else len(seq))
            if n_percent < nThreshold:
                gc_content_value_array.append(gc_content)
                gc_content_value_array_NR.append(gc_content_NR)
                gc_content_value_array_R.append(gc_content_R)
        avg = np.average(gc_content_value_array_R)
        ax.scatter(gc_content_value_array_NR, gc_content_value_array_R,color=colors[j], label=str(name), alpha = 0.7)
        #ax.plot([0,1],[avg,avg], color=colors[index], linestyle="dashed")
        genome = genome+seq
        j+=1
ax.plot([0,1],[0,1],color='black',linestyle="dashed")
avg = GC_content_window_NR(genome)
avgR = GC_content_window_R(genome)
ax.plot([0,1],[avgR,avgR], color='silver', linestyle="dashed", label="Average GC Content of Repeats: "+str(np.round(avgR,3)))
ax.plot([avg,avg],[0,1], color='gold', linestyle="dashed", label="Average GC Content of Non Repeats: "+str(np.round(avg,3)))
ax.set_xlabel('GC Content of Non Repeats')
#ax.set_ylabel('GC Content without Repeats')
ax.set_ylabel('GC Content of Repeats')
ax.legend()
ax.set_title('GC Content Graph for Bfl')
"""

'\n#Add in remove repeats\nfig, ax = plt.subplots(figsize=(8,8))\ncolors = cm.viridis(np.linspace(0,1, numChromosomes))\ngenome = ""\ngcR = []\ngc = []\nj=0\nfor index, row in seqobj.iterrows(): \n        pos_array = []\n        gc_content_value_array = []\n        gc_content_value_array_NR = []\n        gc_content_value_array_R = []\n    \n        n_percent_array = []\n        name = row[\'id\']\n        seq = row[\'seq\']\n        for i in range(0,len(seq),step):\n            subseq = seq[i:i+window]\n            gc_content = (GC_content_window(subseq))\n            gc_content_NR = (GC_content_window_NR(subseq))\n            gc_content_R = (GC_content_window_R(subseq))\n            n_percent = (null_value_window(subseq))\n            #start = (i + 1 if (i+1<=len(seq)) else i)\n            #end = ( i + step if (i+ step<=len(seq)) else len(seq))\n            if n_percent < nThreshold:\n                gc_content_value_array.append(gc_content)\n                gc_content_value_array_NR.

In [6]:
"""
for index, row in seqobj.iterrows(): 
        pos_array = []
        gc_content_value_array = []
        gc_skew_value_array = []
        ag_content_value_array = []
        at_skew_value_array = []

        pos_array = []
        gc_content_value_array_R = []
        gc_skew_value_array_R = []
        ag_content_value_array_R = []
        at_skew_value_array_R = []
        
        n_percent_array = []
        name = row['id']
        seq = row['seq']
        for i in range(0,len(seq),step):
            subseq = seq[i:i+window]
            gc_content = (GC_content_window(subseq))
            gc_skew = (GC_skew_window(subseq))
            ag_content = (AG_content_window(subseq))
            at_skew = (AT_skew_window(subseq))

            gc_content_R = (GC_content_window_R(subseq))
            gc_skew_R = (GC_skew_window_R(subseq))
            ag_content_R = (AG_content_window_R(subseq))
            at_skew_R = (AT_skew_window_R(subseq))
            
            n_percent = (null_value_window(subseq))
            #start = (i + 1 if (i+1<=len(seq)) else i)
            #end = ( i + step if (i+ step<=len(seq)) else len(seq))
            if n_percent < nThreshold:
                pos_array.append(i)
                gc_content_value_array.append(gc_content)
                gc_skew_value_array.append(gc_skew)
                ag_content_value_array.append(ag_content)
                at_skew_value_array.append(at_skew)

                gc_content_value_array_R.append(gc_content_R)
                gc_skew_value_array_R.append(gc_skew_R)
                ag_content_value_array_R.append(ag_content_R)
                at_skew_value_array_R.append(at_skew_R)
                
                n_percent_array.append(n_percent)
        sequence_length = len(seq)
        gene_density = np.zeros(sequence_length)
        filtered_rows = gtf_dataframe.query('sequence == @name and feature == "gene"')
        for start, end in zip(filtered_rows['start'], filtered_rows['end']):
            gene_density[start:end] += 1
        
        max_columns = 2**22
        if sequence_length > max_columns:
            factor = sequence_length // max_columns
            new_length = (sequence_length // factor) * factor  # Ensure new length is a multiple of the factor
            print("Cut off "+str(sequence_length - ((sequence_length // factor) * factor))+" bases")
            gene_density_truncated = gene_density[:new_length]  # Truncate the array
            gene_density_downsampled = gene_density_truncated.reshape(-1, factor).mean(axis=1)
            downsampled = True
        else:
            gene_density_downsampled = gene_density
            downsampled = False
        segments = []
        colors = []
        filteredSC = SC_dataframe.query('sequence == @name')
        for index, row in filteredSC.iterrows():
            start = row['start']
            end = row['end']
            segments.append([(start,0), (end,0)])
            colors.append(hex_to_rgba(row['color']))
        lc = LineCollection(segments, colors=colors, linewidths=50)
        fig, axs = plt.subplots(7,1,figsize=graphShape, sharex=False)
        
        axs[0].plot(pos_array, gc_content_value_array, 'g-', label='GC Content')
        axs[0].plot(pos_array, gc_content_value_array_R, color='DarkOrchid', ls = "--", lw = 1, label='GC Content of Repeats')
        axs[0].set_ylabel('GC Content')
        axs[0].legend()
        axs[0].set_xlim(0, sequence_length)
    
        axs[1].plot(pos_array, gc_skew_value_array, 'r-', label='GC Skew')
        axs[1].plot(pos_array, gc_skew_value_array_R, color='DarkOrchid', ls = "--", lw = 1, label='GC Skew of Repeats')
        axs[1].set_ylabel('GC Skew')
        axs[1].axhline(0,color='black')
        axs[1].legend()
        axs[1].set_xlim(0, sequence_length)
    
        axs[2].plot(pos_array, ag_content_value_array, 'b-', label='AG Content')
        axs[2].plot(pos_array, ag_content_value_array_R, color='DarkOrchid', ls = "--", lw = 1, label='AG Content of Repeats')
        axs[2].set_ylabel('AG Content')
        axs[2].legend()
        axs[2].set_xlim(0, sequence_length)
    
        axs[3].plot(pos_array, at_skew_value_array, 'y-', label='AT Skew')
        axs[3].plot(pos_array, at_skew_value_array_R, color='DarkOrchid', ls = "--", lw = 1, label='AT Skew of Repeats')
        axs[3].set_ylabel('AT Skew')
        axs[3].axhline(0,color='black')
        axs[3].legend()
        axs[3].set_xlim(0, sequence_length)

        axs[4].plot(pos_array, n_percent_array, 'm-', label='Null Percent')
        axs[4].set_ylabel('Null Percent (%)')
        axs[4].legend()
        axs[4].set_xlabel('Position in Sequence (bp)')
        axs[4].set_xlim(0, sequence_length)

        axs[5].add_collection(lc)
        for i in range(0,len(segments)):
            axs[5].scatter((segments[i][0][0]+segments[i][1][0])/2,0,color=colors[i])
        axs[5].set_ylim(-0.5,0.5)
        axs[5].set_xlim(0, sequence_length)
        axs[5].set_ylabel('SC Orthologues')
    
        norm=plt.Normalize(gene_density_downsampled.min(), gene_density_downsampled.max())
        cmap = plt.get_cmap('viridis')
        axs[6].imshow(gene_density_downsampled[np.newaxis, :], aspect='auto', cmap=cmap, norm=norm)
        axs[6].set_yticks([])
        axs[6].set_ylabel('Gene Density')
        
        if downsampled:
            tick_positions = np.linspace(0, len(gene_density_downsampled), num=10, dtype=int)
            tick_labels = np.linspace(0, sequence_length, num=10, dtype=int)
            axs[6].set_xticks(tick_positions)
            axs[6].set_xticklabels(tick_labels)
        else:
            axs[6].set_xlim(0, sequence_length)
    
        #Colorbar
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = fig.colorbar(sm, ax=axs[6], orientation='horizontal', pad=0.2)
        #cbar.set_label('Gene Density')
    
        fig.suptitle("Sliding Window Analysis for different metrics on the "+str(name)+" scaffold")
        plt.subplots_adjust(hspace=0.5)
        #plt.tight_layout()
        #plt.subplots_adjust(top=0.95)
        plt.show()
"""

'\nfor index, row in seqobj.iterrows(): \n        pos_array = []\n        gc_content_value_array = []\n        gc_skew_value_array = []\n        ag_content_value_array = []\n        at_skew_value_array = []\n\n        pos_array = []\n        gc_content_value_array_R = []\n        gc_skew_value_array_R = []\n        ag_content_value_array_R = []\n        at_skew_value_array_R = []\n        \n        n_percent_array = []\n        name = row[\'id\']\n        seq = row[\'seq\']\n        for i in range(0,len(seq),step):\n            subseq = seq[i:i+window]\n            gc_content = (GC_content_window(subseq))\n            gc_skew = (GC_skew_window(subseq))\n            ag_content = (AG_content_window(subseq))\n            at_skew = (AT_skew_window(subseq))\n\n            gc_content_R = (GC_content_window_R(subseq))\n            gc_skew_R = (GC_skew_window_R(subseq))\n            ag_content_R = (AG_content_window_R(subseq))\n            at_skew_R = (AT_skew_window_R(subseq))\n            \n

In [7]:

numChromosomes = 19
filename = 'input_data/genomes/Llo.fna'
#filename = 'input_data/genomes/BflAlt.fna'
gtffile = 'input_data/gene_rows/Llo.gtf'
SCfile = 'input_data/gene_rows_SC/Llo_coordinates.tsv'

records = SeqIO.parse(filename, 'fasta')
data = [(record.id, str(record.seq), len(record.seq)) for record in records]
seqobj = pd.DataFrame(data, columns=['id', 'seq', 'seqlen'])
seqobj = seqobj.sort_values('seqlen', ascending=False).head(numChromosomes)

columns = ["sequence", "source", "feature", "start", "end", "score", "strand", "frame", "attribute"]
gtf_dataframe = pd.read_csv(gtffile, sep = '\t', comment = '#', header = None, names = columns, dtype={'start': int, 'end': int})

SCcolumns = ["index", "Junk", "sequence", "start", "end", "color_key"]
SC_dataframe = pd.read_csv(SCfile, sep = '\t', comment = '#', header = None, names = SCcolumns, dtype={'start': int, 'end': int})
SC_dataframe = SC_dataframe[SC_dataframe['color_key'].isin(["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S"])]
SC_dataframe['color'] = SC_dataframe['color_key'].map(color_mapping)


In [ ]:

#Just for investigating the bimodal distribution in Llo - centromeres
for index, row in seqobj.query('id == "NC_088313.1"').iterrows(): 
        pos_array = []
        gc_content_value_array = []
        n_percent_array = []
        name = row['id']
        seq = row['seq']
        for i in range(0,len(seq),step):
            subseq = seq[i:i+window]
            gc_content = (GC_content_window(subseq))
            n_percent = (null_value_window(subseq))
            if n_percent < nThreshold:
                pos_array.append(i)
                gc_content_value_array.append(gc_content)
                n_percent_array.append(n_percent)
        
        sequence_length = len(seq)
        gene_density = np.zeros(sequence_length)
        filtered_rows = gtf_dataframe.query('sequence == @name and feature == "gene"')
        for start, end in zip(filtered_rows['start'], filtered_rows['end']):
            gene_density[start:end] += 1
        
        max_columns = 2**22
        if sequence_length > max_columns:
            factor = sequence_length // max_columns
            new_length = (sequence_length // factor) * factor  # Ensure new length is a multiple of the factor
            print("Cut off "+str(sequence_length - ((sequence_length // factor) * factor))+" bases")
            gene_density_truncated = gene_density[:new_length]  # Truncate the array
            gene_density_downsampled = gene_density_truncated.reshape(-1, factor).mean(axis=1)
            downsampled = True
        else:
            gene_density_downsampled = gene_density
            downsampled = False
        segments = []
        colors = []
        filteredSC = SC_dataframe.query('sequence == @name')
        for index, row in filteredSC.iterrows():
            start = row['start']
            end = row['end']
            segments.append([(start,0), (end,0)])
            colors.append(hex_to_rgba(row['color']))
        lc = LineCollection(segments, colors=colors, linewidths=50)
        fig, axs = plt.subplots(4,1,figsize=graphShape, sharex=False)
        
        axs[0].plot(pos_array, gc_content_value_array, 'g-', label='GC Content')
        axs[0].set_ylabel('GC Content')
        axs[0].legend()
        axs[0].set_xlim(0, sequence_length)

        for start, end in zip(filtered_rows['start'],filtered_rows['end']):
            gcctent = GC_content_window(seq[start:end])
            if gcctent > 0.5 and gcctent < 0.6:
                axs[1].scatter((start+end)/2,0)
        axs[1].set_ylabel('Between 0.5 to 0.6')
        axs[1].set_xlim(0, sequence_length)

        axs[2].add_collection(lc)
        for i in range(0,len(segments)):
            axs[2].scatter((segments[i][0][0]+segments[i][1][0])/2,0,color=colors[i])
        axs[2].set_ylim(-0.5,0.5)
        axs[2].set_xlim(0, sequence_length)
        axs[2].set_ylabel('SC Orthologues')
    
        norm=plt.Normalize(gene_density_downsampled.min(), gene_density_downsampled.max())
        cmap = plt.get_cmap('viridis')
        axs[3].imshow(gene_density_downsampled[np.newaxis, :], aspect='auto', cmap=cmap, norm=norm)
        axs[3].set_yticks([])
        axs[3].set_ylabel('Gene Density')
        
        if downsampled:
            tick_positions = np.linspace(0, len(gene_density_downsampled), num=10, dtype=int)
            tick_labels = np.linspace(0, sequence_length, num=10, dtype=int)
            axs[3].set_xticks(tick_positions)
            axs[3].set_xticklabels(tick_labels)
        else:
            axs[3].set_xlim(0, sequence_length)
    
        #Colorbar
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = fig.colorbar(sm, ax=axs[3], orientation='horizontal', pad=0.2)
        #cbar.set_label('Gene Density')
    
        fig.suptitle("Sliding Window Analysis for different metrics on the "+str(name)+" scaffold")
        plt.subplots_adjust(hspace=0.5)
        #plt.tight_layout()
        #plt.subplots_adjust(top=0.95)
        plt.show()
